In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2-4*x+5

In [ ]:
f(3.0)

In [ ]:
xs=np.arange(-5,5,0.25)#从-5到5(不含)步长0.25
ys=f(xs)
plt.plot(xs,ys);

In [ ]:
h=0.000001
x=-3
(f(x+h)-f(x))/h

In [ ]:
a=2.0
b=-3.0
c=10.0
d=a*b+c
print(d)

In [ ]:
h=0.0001

a=2.0
b=-3.0
c=10.0

d1=a*b+c
b+=h
d2=a*b+c

print('d1',d1)
print('d2',d2)
print('slope',(d1-d2)/h)

In [ ]:
class Value:

    def __init__(self,data,_children=(),_op='',label=''):#初始化class,children是一个空tuple
        self.data=data
        self.grad=0.0
        self._backward=lambda:None#默认是空函数,不做任何操作
        self._prev=set(_children)#把元组转换为集合
        self._op=_op
        self.label=label

    def __repr__(self):
        return f"Value(data={self.data})"#wrapper函数提供一个更加美观的打印结果
    
    def __add__(self,other):
        out=Value(self.data+other.data,(self,other),'+')
        return out
    
    def __mul__(self,other):
        out=Value(self.data*other.data,(self,other),'*')
        return out
    
    def tanh(self):
        x=self.data
        t=(math.exp(2*x)-1)/(math.exp(2*x)+1)
        out=Value(t,(self,),'tanh')
        return out
    
a=Value(2.0,label='a')
b=Value(-3.0,label='b')
c=Value(10.0,label='c')
e=a*b; e.label='e'
d=e+c;d.label='d'
f=Value(-2.0,label='f')
L=f*d;L.label='L'
#(a.__mul__(b)).__add__(c)

In [ ]:
d._prev

In [ ]:
d._op

In [ ]:
a._op

In [ ]:
from graphviz import Digraph

def trace(root):
    #构建计算图中的所有边和节点
    nodes,edges=set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes,edges

def draw_dot(root):
    dot=Digraph(format='svg',graph_attr={'rankdir':'LR'})#LR:left to right
    nodes,edges=trace(root)
    for n in nodes:
        uid=str(id(n))
        dot.node(name=uid,label="{%s|data %.4f|grad %.4f}"%(n.label,n.data,n.grad),shape='record')
        if n._op:
            dot.node(name=uid+n._op,label=n._op)# 创建操作节点
            dot.edge(uid+n._op,uid)#操作节点 → 数值节点

    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2))+n2._op)# 输入节点→操作节点

    return dot

In [ ]:
L.grad=1.0
f.grad=4
d.grad=-2
c.grad=-2
e.grad=-2
b.grad=-4
a.grad=6

In [ ]:
draw_dot(L)#forward pass

In [ ]:
a.data+=0.01*a.grad#向梯度方向微调,可以使L正向增大,0.01就是步长
b.data+=0.01*b.grad
c.data+=0.01*c.grad
f.data+=0.01*f.grad

e=a*b
d=c+e
L=f*d

print(L.data)

In [ ]:
#反向传播
def lol():
    h=0.0001

    a=Value(2.0,label='a')
    b=Value(-3.0,label='b')
    c=Value(10.0,label='c')
    e=a*b; e.label='e'
    d=e+c;d.label='d'
    f=Value(-2.0,label='f')
    L=f*d;L.label='L'
    L1=L.data

    a=Value(2.0,label='a')
    b=Value(-3.0,label='b')
    c=Value(10.0,label='c')
    e=a*b; e.label='e'
    d=e+c;d.label='d'
    d.data+=h
    f=Value(-2.0,label='f')
    L=f*d;L.label='L'
    L2=L.data

    print((L2-L1)/h)

lol()

In [ ]:
plt.plot(np.arange(-5,5,0.2),np.tanh(np.arange(-5,5,0.2)));plt.grid();

In [ ]:
#inputs x1,x2
x1=Value(2.0,label='x1')
x2=Value(0.0,label='x2')
#weights w1,w2
w1=Value(-3.0,label='w1')
w2=Value(1.0,label='w2')
#bias of the neuron
b=Value(6.8813735870195432,label='b')
#x1*w1 +x2*w2 + b
x1w1=x1*w1;x1w1.label='x1*w1'
x2w2=x2*w2;x2w2.label='x2*w2'
x1w1x2w2=x1w1+x2w2;x1w1x2w2.label='x1*w1 + x2*w2'
n=x1w1x2w2+b;n.label='n'
o=n.tanh();o.label='o'

In [ ]:
o.grad=1.0

In [ ]:
#o=tanh(n)
#do/dn=1-o**2
1-o.data**2
n.grad=0.5
b.grad=0.5
x1w1x2w2.grad=0.5

In [ ]:
x1w1.grad=0.5
x2w2.grad=0.5

In [ ]:
w1.grad=x1.data*x1w1.grad
x1.grad=w1.data*x1w1.grad
w2.grad=x2.data*x2w2.grad
x2.grad=w2.data*x2w2.grad

In [ ]:
draw_dot(o)